<a href="https://colab.research.google.com/github/Slother12/Credit-card-fault-detection/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data loading

In [5]:
credit_card_data = pd.read_csv('/creditcard.csv')

In [6]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [7]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
39697,39927,-1.466679,2.425732,0.877724,3.950765,0.762477,1.162748,0.575625,-0.288354,-1.357990,2.907686,-0.890614,-0.462249,1.291626,-0.392713,1.434131,-0.347514,-0.124999,-0.270457,0.875010,0.129962,0.186051,0.571010,-0.107517,-0.754306,-0.752828,0.113179,-2.079421,-0.809173,1.50,0.0
39698,39927,-0.523165,-0.100021,0.892966,-1.900405,-0.156870,-0.783894,0.917683,-0.308345,-1.305284,0.514227,0.675557,-1.629259,-2.491738,0.386368,-0.595120,1.352518,-0.228732,-0.936432,0.660506,-0.071640,-0.082504,-0.414677,-0.063392,-0.087455,-0.303383,-0.682889,-0.178417,-0.137169,100.92,0.0
39699,39928,-2.768425,-1.007072,2.151127,0.117797,1.283178,1.869731,-0.562240,0.820374,0.348797,-0.122472,1.700394,1.375019,0.623614,-0.610102,0.577459,-0.941449,0.571209,-1.502828,-0.785017,-0.491556,-0.182963,0.778210,0.904077,-1.288631,0.212441,0.483975,-0.027614,-0.582813,11.99,0.0
39700,39928,1.201327,0.158614,-0.325263,0.471667,0.086446,-0.770357,0.422151,-0.205277,-0.451865,0.161808,1.063432,0.650581,-0.207706,0.742138,0.054512,0.176587,-0.598347,0.093661,0.320508,-0.023074,0.027664,-0.018485,-0.199382,0.053605,0.683829,0.428416,-0.077342,-0.006394,45.00,0.0
39701,39929,1.097669,-1.315782,0.659681,-0.683915,-1.342612,0.332629,-1.110900,0.194811,-0.248825,0.624948,0.324492,-0.279047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39702 entries, 0 to 39701
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    39702 non-null  int64  
 1   V1      39702 non-null  float64
 2   V2      39702 non-null  float64
 3   V3      39702 non-null  float64
 4   V4      39702 non-null  float64
 5   V5      39702 non-null  float64
 6   V6      39702 non-null  float64
 7   V7      39702 non-null  float64
 8   V8      39702 non-null  float64
 9   V9      39702 non-null  float64
 10  V10     39702 non-null  float64
 11  V11     39702 non-null  float64
 12  V12     39702 non-null  float64
 13  V13     39701 non-null  float64
 14  V14     39701 non-null  float64
 15  V15     39701 non-null  float64
 16  V16     39701 non-null  float64
 17  V17     39701 non-null  float64
 18  V18     39701 non-null  float64
 19  V19     39701 non-null  float64
 20  V20     39701 non-null  float64
 21  V21     39701 non-null  float64
 22

In [9]:
#checking number of missing values
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [12]:
credit_card_data.shape

(39702, 31)

In [13]:
credit_card_data = credit_card_data.dropna(how = 'any')

In [14]:
credit_card_data.shape

(39701, 31)

In [15]:
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


now our data is not missing

In [16]:
credit_card_data['Class'].value_counts()

,count
Class,
0.0,39597
1.0,104


dataset is highly unbalanced\
0 --> normal
1-->fraud

In [17]:
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [18]:
print(legit.shape)
print(fraud.shape)

(39597, 31)
(104, 31)


In [19]:
#statistical data interpretation for making the data balanced
legit.Amount.describe()

,Amount
count,39597.000000
mean,87.420700
std,234.514383
min,0.000000
25%,7.420000
50%,23.500000
75%,79.000000
max,7879.420000


In [20]:
fraud.Amount.describe()

,Amount
count,104.000000
mean,97.070769
std,255.012160
min,0.000000
25%,1.000000
50%,3.775000
75%,99.990000
max,1809.680000


In [21]:
#comparing value of bth transaction
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,25514.725686,-0.194310,0.036967,0.744162,0.174545,-0.217414,0.107151,-0.093182,0.031783,0.222434,-0.054193,0.409491,-0.445757,0.247772,0.260637,0.097049,0.010676,0.171875,-0.082776,-0.034697,0.046047,-0.031386,-0.111706,-0.039470,0.007851,0.135895,0.022779,0.005987,0.003920,87.420700
1.0,20683.201923,-7.697309,5.766703,-10.853666,5.865850,-5.424996,-2.275717,-7.641867,3.827809,-2.924282,-6.655401,5.450235,-8.328284,0.398577,-8.293300,-0.018106,-4.914592,-7.959096,-2.830897,0.486825,0.663205,0.626795,-0.345972,-0.342944,-0.230514,0.299242,0.176522,0.811431,0.099914,97.070769


In [22]:
legit_sample = legit.sample(n=104)

In [23]:
new_dataset = pd.concat([legit_sample,fraud],axis=0)

In [24]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
2363,1907,-1.370659,-0.779180,1.950071,1.253887,0.698670,0.411538,-0.389679,0.409366,0.126990,-0.426380,-1.412800,-0.325740,-0.027579,-0.210928,1.130063,-0.129164,-0.150760,0.405565,0.295897,0.558772,0.353333,0.666588,0.179075,-0.772127,0.034107,-0.106901,0.113362,0.157764,130.00,0.0
8937,12287,-0.676165,0.569624,1.873835,-1.065547,0.317526,-0.561320,0.286973,-0.002883,0.827354,-1.088624,1.932262,-1.818893,1.943295,1.552771,-0.808391,0.777134,-0.242732,0.527902,-0.239734,-0.019310,-0.142180,-0.312190,-0.108880,0.002219,-0.305153,0.714957,-0.031673,0.086366,4.52,0.0
29907,35689,-1.014675,0.407006,1.767618,-1.948179,-0.226589,-1.096734,0.769193,0.007968,1.151378,-1.774272,-0.837687,0.202763,-1.017631,-0.038837,-0.125399,-0.709218,-0.093561,-0.242349,0.182150,0.044762,-0.231749,-0.524090,-0.205566,0.354901,0.716011,-0.946873,0.298499,0.129921,38.21,0.0
22978,32528,1.577636,-1.106899,0.107291,-1.650423,-1.099827,-0.124540,-1.134309,-0.053471,-1.550219,1.429079,-1.786365,-1.637993,0.150648,-0.337201,0.927370,0.250771,-0.047440,0.452154,0.288736,-0.294310,-0.461002,-1.026208,0.003828,-1.079039,0.292828,-0.322730,0.034851,0.016415,25.00,0.0
3718,3198,-0.750999,-1.105777,1.737505,-1.493267,-0.218925,0.341096,-0.239610,-0.008562,-2.995626,1.406431,1.777568,-0.326635,0.751652,-0.261651,0.724653,-1.501273,1.152347,-0.180425,0.498956,0.294764,0.074248,0.501449,0.054880,-0.275861,0.186399,0.074702,-0.099197,-0.149000,110.75,0.0


In [25]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
30473,35942,-4.194074,4.382897,-5.118363,4.455230,-4.812621,-1.224645,-7.281328,3.332250,-3.679659,-7.524368,2.954344,-7.099825,1.520369,-7.687803,-0.225002,-8.520850,-13.277300,-5.253705,3.623332,0.579098,1.550473,0.614573,0.028521,0.013704,-0.149512,-0.131687,0.473934,0.473757,14.46,1.0
30496,35953,-4.844372,5.649439,-6.730396,5.252842,-4.409566,-1.740767,-6.311699,3.449167,-5.416284,-7.833556,3.657350,-7.781448,1.507720,-8.604759,0.010593,-6.553973,-12.624394,-4.989910,1.645414,0.284555,1.194888,-0.845753,0.190674,-0.216443,-0.325033,-0.270328,0.210214,0.391855,111.70,1.0
31002,36170,-5.685013,5.776516,-7.064977,5.902715,-4.715564,-1.755633,-6.958679,3.877795,-5.541529,-7.502112,3.676703,-7.642983,1.689865,-8.299660,0.646996,-6.792795,-12.075932,-4.882180,2.626625,0.299489,1.128641,-0.962960,-0.110045,-0.177733,-0.089175,-0.049447,0.303445,0.219380,111.70,1.0
33276,37167,-7.923891,-5.198360,-3.000024,4.420666,2.272194,-3.394483,-5.283435,0.131619,0.658176,-0.794994,3.266066,-2.719185,-0.124104,-5.274866,0.638575,-2.995830,-4.698433,-1.711871,3.025261,-2.169811,-0.734308,-0.599926,-4.908301,0.410170,-1.167660,0.520508,1.937421,-1.552593,12.31,1.0
39183,39729,-0.964567,-1.643541,-0.187727,1.158253,-2.458336,0.852222,2.785163,-0.303609,0.940006,-1.965309,0.159744,-0.490697,-1.181977,-1.958876,1.152743,-1.341269,2.498325,0.777831,1.406045,1.784449,0.447180,0.536204,1.634061,0.203839,0.218749,-0.221886,-0.308555,-0.164500,776.83,1.0


In [26]:
new_dataset['Class'].value_counts()

,count
Class,
0.0,104
1.0,104


In [27]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,26245.009615,-0.183935,0.143569,0.786230,0.022295,-0.139239,0.253551,-0.244399,-0.409730,0.126519,-0.164199,0.494601,-0.427334,0.286143,0.368109,0.161461,0.013802,0.171219,-0.032377,-0.016172,0.093291,-0.217725,-0.047407,0.045666,-0.013830,0.092366,0.106394,0.020608,0.004530,68.551635
1.0,20683.201923,-7.697309,5.766703,-10.853666,5.865850,-5.424996,-2.275717,-7.641867,3.827809,-2.924282,-6.655401,5.450235,-8.328284,0.398577,-8.293300,-0.018106,-4.914592,-7.959096,-2.830897,0.486825,0.663205,0.626795,-0.345972,-0.342944,-0.230514,0.299242,0.176522,0.811431,0.099914,97.070769


In [28]:
X = new_dataset.drop(columns='Class',axis =1)
Y= new_dataset['Class']

In [29]:
print(X)

        Time        V1        V2        V3  ...       V26       V27       V28  Amount
2363    1907 -1.370659 -0.779180  1.950071  ... -0.106901  0.113362  0.157764  130.00
8937   12287 -0.676165  0.569624  1.873835  ...  0.714957 -0.031673  0.086366    4.52
29907  35689 -1.014675  0.407006  1.767618  ... -0.946873  0.298499  0.129921   38.21
22978  32528  1.577636 -1.106899  0.107291  ... -0.322730  0.034851  0.016415   25.00
3718    3198 -0.750999 -1.105777  1.737505  ...  0.074702 -0.099197 -0.149000  110.75
...      ...       ...       ...       ...  ...       ...       ...       ...     ...
30473  35942 -4.194074  4.382897 -5.118363  ... -0.131687  0.473934  0.473757   14.46
30496  35953 -4.844372  5.649439 -6.730396  ... -0.270328  0.210214  0.391855  111.70
31002  36170 -5.685013  5.776516 -7.064977  ... -0.049447  0.303445  0.219380  111.70
33276  37167 -7.923891 -5.198360 -3.000024  ...  0.520508  1.937421 -1.552593   12.31
39183  39729 -0.964567 -1.643541 -0.187727  ... -0.221

In [30]:
print(Y)

2363     0.0
8937     0.0
29907    0.0
22978    0.0
3718     0.0
        ... 
30473    1.0
30496    1.0
31002    1.0
33276    1.0
39183    1.0
Name: Class, Length: 208, dtype: float64


In [33]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [34]:
print(X.shape,X_train.shape,X_test.shape)

(208, 30) (166, 30) (42, 30)


Logistic regression

In [38]:
model = LogisticRegression(solver='liblinear', max_iter=500)
model.fit(X_train, Y_train)


LogisticRegression(max_iter=500, solver='liblinear')

Model evaluation

Accuracy Score

In [40]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction , Y_train)

In [41]:
print('accuracy on training data : ', training_data_accuracy)

accuracy on training data :  0.9698795180722891


In [43]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [44]:
print('accuracy on test data : ', test_data_accuracy)

accuracy on test data :  0.9285714285714286
